Import the Data

In [1]:
import pickle

# Define the file path
file_path = 'C:/Users/lanza/Master_Thesis_EL/Integrated-vs-Seperated-Master-Thesis/hpc_mount/dataset_list.pkl'

# Open the file in read-binary mode
with open(file_path, 'rb') as file:
    # Load the dictionary
    data_dict = pickle.load(file)

# Print the dictionary

for dict in data_dict:
    print(dict)

{'dataset_id': 'set_530650', 'dataset_path': '/pfs/work7/workspace/scratch/ma_elanza-thesislanza/set_530650/set_530650_data.h5', 'folder_path': '/pfs/work7/workspace/scratch/ma_elanza-thesislanza/set_530650'}
{'dataset_id': 'set_5306100', 'dataset_path': '/pfs/work7/workspace/scratch/ma_elanza-thesislanza/set_5306100/set_5306100_data.h5', 'folder_path': '/pfs/work7/workspace/scratch/ma_elanza-thesislanza/set_5306100'}
{'dataset_id': 'set_5306150', 'dataset_path': '/pfs/work7/workspace/scratch/ma_elanza-thesislanza/set_5306150/set_5306150_data.h5', 'folder_path': '/pfs/work7/workspace/scratch/ma_elanza-thesislanza/set_5306150'}
{'dataset_id': 'set_5306200', 'dataset_path': '/pfs/work7/workspace/scratch/ma_elanza-thesislanza/set_5306200/set_5306200_data.h5', 'folder_path': '/pfs/work7/workspace/scratch/ma_elanza-thesislanza/set_5306200'}
{'dataset_id': 'set_5306250', 'dataset_path': '/pfs/work7/workspace/scratch/ma_elanza-thesislanza/set_5306250/set_5306250_data.h5', 'folder_path': '/pfs

Define the cost strucutre and calculate optimal profit

In [2]:
import numpy as np
# Parameters for multi-item newsvendor problem
prices = np.array([0.3, 0.5, 0.6, 0.5, 0.5, 0.5]) #price data
prices = prices.reshape(6,1)
costs = np.array([0.06, 0.06, 0.06, 0.06, 0.06, 0.06]) #cost data
costs = costs.reshape(6,1)
salvages = np.array([0.01, 0.01, 0.01, 0.01, 0.01, 0.01]) #salvage data
salvages = salvages.reshape(6,1)
underage_data = prices - costs 
overage_data = costs - salvages 
underage_data_single = underage_data[0,0]
overage_data_single = overage_data[0,0]



alpha_data = np.array([             #alpha data
    [0.0, 0.1, 0.05, 0.1, 0.05, 0.1],
    [0.15, 0.0, 0.1, 0.05, 0.05, 0.05],
    [0.1, 0.2, 0.0, 0.05, 0.1, 0.05],
    [0.05, 0.05, 0.05, 0.0, 0.15, 0.2],
    [0.1, 0.05, 0.15, 0.2, 0.0, 0.05],
    [0.05, 0.1, 0.05, 0.15, 0.1, 0.0]
])

In [3]:
from IvsS_Utils import load_generated_data, load_cost_structure, nvps_profit

# Max Profit for multi-item newsvendor problem
X_train, y_train, X_val, y_val, X_test, y_test = load_generated_data(path='C:/Users/lanza/Master_Thesis_EL/Integrated-vs-Seperated-Master-Thesis/hpc_mount/set_130650_data.h5', multi=True)
load_cost_structure(alpha_input=alpha_data, underage_input=underage_data, overage_input=overage_data)
max_profit_multi_normal = nvps_profit(y_test,y_test)
print(max_profit_multi_normal)

# Max Profit for single-item newsvendor problem
X_train, y_train, X_val, y_val, X_test, y_test = load_generated_data(path='C:/Users/lanza/Master_Thesis_EL/Integrated-vs-Seperated-Master-Thesis/hpc_mount/set_130650_data.h5', multi=False)
load_cost_structure(alpha_input=alpha_data, underage_input=underage_data_single, overage_input=overage_data_single)
max_profit_single_normal = nvps_profit(y_test,y_test)
print(max_profit_single_normal)

# Max Profit for variance increase
X_train, y_train, X_val, y_val, X_test, y_test = load_generated_data(path='C:/Users/lanza/Master_Thesis_EL/Integrated-vs-Seperated-Master-Thesis/test/set_5306100_data.h5', multi=False)
max_profit_single_var_100 = nvps_profit(y_test,y_test)
variance_100 = np.var(y_test)
X_train, y_train, X_val, y_val, X_test, y_test = load_generated_data(path='C:/Users/lanza/Master_Thesis_EL/Integrated-vs-Seperated-Master-Thesis/test/set_5306150_data.h5', multi=False)
max_profit_single_var_150 = nvps_profit(y_test,y_test)
variance_150 = np.var(y_test)
X_train, y_train, X_val, y_val, X_test, y_test = load_generated_data(path='C:/Users/lanza/Master_Thesis_EL/Integrated-vs-Seperated-Master-Thesis/test/set_5306200_data.h5', multi=False)
max_profit_single_var_200 = nvps_profit(y_test,y_test)
variance_200 = np.var(y_test)
X_train, y_train, X_val, y_val, X_test, y_test = load_generated_data(path='C:/Users/lanza/Master_Thesis_EL/Integrated-vs-Seperated-Master-Thesis/test/set_5306250_data.h5', multi=False)
max_profit_single_var_250 = nvps_profit(y_test,y_test)
variance_250 = np.var(y_test)
X_train, y_train, X_val, y_val, X_test, y_test = load_generated_data(path='C:/Users/lanza/Master_Thesis_EL/Integrated-vs-Seperated-Master-Thesis/test/set_5306300_data.h5', multi=False)
max_profit_single_var_300 = nvps_profit(y_test,y_test) 
variance_300 = np.var(y_test)

# Max Profit for heterogenity
X_train, y_train, X_val, y_val, X_test, y_test = load_generated_data(path='C:/Users/lanza/Master_Thesis_EL/Integrated-vs-Seperated-Master-Thesis/test/set_4306510_data.h5', multi=False)
max_profit_single_het_10 = nvps_profit(y_test,y_test)
variance_het_10 = np.var(y_test)
X_train, y_train, X_val, y_val, X_test, y_test = load_generated_data(path='C:/Users/lanza/Master_Thesis_EL/Integrated-vs-Seperated-Master-Thesis/test/set_4306520_data.h5', multi=False)
max_profit_single_het_20 = nvps_profit(y_test,y_test)
variance_het_20 = np.var(y_test)
X_train, y_train, X_val, y_val, X_test, y_test = load_generated_data(path='C:/Users/lanza/Master_Thesis_EL/Integrated-vs-Seperated-Master-Thesis/test/set_4306530_data.h5', multi=False)
max_profit_single_het_30 = nvps_profit(y_test,y_test)
variance_het_30 = np.var(y_test)   
X_train, y_train, X_val, y_val, X_test, y_test = load_generated_data(path='C:/Users/lanza/Master_Thesis_EL/Integrated-vs-Seperated-Master-Thesis/test/set_4306540_data.h5', multi=False)
max_profit_single_het_40 = nvps_profit(y_test,y_test)
variance_het_40 = np.var(y_test)
X_train, y_train, X_val, y_val, X_test, y_test = load_generated_data(path='C:/Users/lanza/Master_Thesis_EL/Integrated-vs-Seperated-Master-Thesis/test/set_4306550_data.h5', multi=False)
max_profit_single_het_50 = nvps_profit(y_test,y_test)
variance_het_50 = np.var(y_test)



c:\Users\lanza\.pyenv\pyenv-win\versions\3.11.7\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


63578.88
5979.119999999999


Extract information from data

In [4]:
import os
import pickle
import pandas as pd

def load_metadata(directory):
    metadata = []
    for file_name in os.listdir(directory):
        if file_name.endswith('meta.pkl'):
            with open(os.path.join(directory, file_name), 'rb') as f:
                data = pickle.load(f)
                metadata.append({
                    'file_name': file_name,
                    'hyperparameter': data['hyperparameter'],
                    'profit': data['profit'],
                    'elapsed_time': data['elapsed_time'],
                    'peak_memory' : data['peak_memory'],
                    'avg_memory': data['avg_memory']
                })
    return metadata

# Directory where the test files are located
directory_path = 'C:/Users/lanza/Master_Thesis_EL/Integrated-vs-Seperated-Master-Thesis/hpc_mount/'

# Load metadata from all models
metadata_list = load_metadata(directory_path)

# Convert to DataFrame for better visualization
df_metadata = pd.DataFrame(metadata_list)



In [5]:
print(df_metadata.head())   

                               file_name  \
0    set_130650_ANN_complex_IOA_meta.pkl   
1  set_130650_ANN_complex_SOAnp_meta.pkl   
2   set_130650_ANN_complex_SOAp_meta.pkl   
3     set_130650_ANN_simple_IOA_meta.pkl   
4   set_130650_ANN_simple_SOAnp_meta.pkl   

                                    hyperparameter        profit  \
0   [2, 19, 0.03259591854253751, 42, 10, 16, relu]  52604.612790   
1  [8, 15, 0.015497711938645823, 18, 10, 16, relu]  61152.380547   
2  [8, 15, 0.015497711938645823, 18, 10, 16, relu]  12980.912077   
3    [7, 8, 0.09986941432760484, 50, 10, 16, relu]   5215.587519   
4   [6, 15, 0.09117597957701895, 29, 10, 16, relu]   5626.292373   

   elapsed_time  peak_memory   avg_memory  
0    161.687909  1203.960938  1002.794437  
1    276.144137  1209.804688  1062.006270  
2    303.158436  1209.804688  1062.006270  
3    333.363864  2368.179688  1734.291984  
4    429.537078  3876.964844  3520.656250  


In [6]:
# Corrected code: Specify the full path including the file name and extension
df_metadata.to_csv("C:/Users/lanza/Master_Thesis_EL/Integrated-vs-Seperated-Master-Thesis/Main/metadata.csv")

In [7]:
df_metadata.loc[df_metadata['file_name'] == 'set_330650_ANN_complex_IOA_meta.pkl', 'profit'] = 0
df_metadata.loc[df_metadata['file_name'] == 'set_230650_ANN_complex_IOA_meta.pkl', 'profit'] = 0

In [8]:
df_simple = df_metadata[df_metadata['file_name'].str.contains('simple')]
df_complex = df_metadata[df_metadata['file_name'].str.contains('complex')]

df_simple['relative_profit'] = df_metadata['profit'] / max_profit_single_normal
df_complex['relative_profit'] = df_metadata['profit'] / max_profit_multi_normal

df_simple = df_simple.sort_values(by='relative_profit', ascending=True)
df_complex = df_complex.sort_values(by='relative_profit', ascending=True)

C:\Users\lanza\AppData\Local\Temp\ipykernel_8932\1547034099.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_simple['relative_profit'] = df_metadata['profit'] / max_profit_single_normal
C:\Users\lanza\AppData\Local\Temp\ipykernel_8932\1547034099.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_complex['relative_profit'] = df_metadata['profit'] / max_profit_multi_normal


In [9]:
df_simple.loc[df_simple['file_name'].str.contains('5306100'), 'relative_profit'] = df_simple['profit'] / max_profit_single_var_100
df_simple.loc[df_simple['file_name'].str.contains('5306150'), 'relative_profit'] = df_simple['profit'] / max_profit_single_var_150
df_simple.loc[df_simple['file_name'].str.contains('5306200'), 'relative_profit'] = df_simple['profit'] / max_profit_single_var_200
df_simple.loc[df_simple['file_name'].str.contains('5306250'), 'relative_profit'] = df_simple['profit'] / max_profit_single_var_250
df_simple.loc[df_simple['file_name'].str.contains('5306300'), 'relative_profit'] = df_simple['profit'] / max_profit_single_var_300

df_simple.loc[df_simple['file_name'].str.contains('4306510'), 'relative_profit'] = df_simple['profit'] / max_profit_single_het_10
df_simple.loc[df_simple['file_name'].str.contains('4306520'), 'relative_profit'] = df_simple['profit'] / max_profit_single_het_20
df_simple.loc[df_simple['file_name'].str.contains('4306530'), 'relative_profit'] = df_simple['profit'] / max_profit_single_het_30
df_simple.loc[df_simple['file_name'].str.contains('4306540'), 'relative_profit'] = df_simple['profit'] / max_profit_single_het_40
df_simple.loc[df_simple['file_name'].str.contains('4306550'), 'relative_profit'] = df_simple['profit'] / max_profit_single_het_50


In [10]:
# Initialize DataFrame with the correct dtype for relative_profit values
df_test = pd.DataFrame(0.0, index=range(5), columns=['ann_ioa', 'ann_soap', 'ann_soanp', 'xgb_ioa', 'xgb_soap', 'xgb_soanp'])

# Use .loc for assignments and ensure values are cast to float
for i in range(5):
    df_test.loc[i, 'ann_ioa'] = df_simple[df_simple['file_name'].str.contains(f'{i+1}30650_ANN_simple_IOA')]['relative_profit'].astype(float).values
    df_test.loc[i, 'ann_soap'] = df_simple[df_simple['file_name'].str.contains(f'{i+1}30650_ANN_simple_SOAp')]['relative_profit'].astype(float).values
    df_test.loc[i, 'ann_soanp'] = df_simple[df_simple['file_name'].str.contains(f'{i+1}30650_ANN_simple_SOAnp')]['relative_profit'].astype(float).values
    df_test.loc[i, 'xgb_ioa'] = df_simple[df_simple['file_name'].str.contains(f'{i+1}30650_XGB_simple_IOA')]['relative_profit'].astype(float).values
    df_test.loc[i, 'xgb_soap'] = df_simple[df_simple['file_name'].str.contains(f'{i+1}30650_XGB_simple_SOAp')]['relative_profit'].astype(float).values
    df_test.loc[i, 'xgb_soanp'] = df_simple[df_simple['file_name'].str.contains(f'{i+1}30650_XGB_simple_SOAnp')]['relative_profit'].astype(float).values

print(df_test)

    ann_ioa  ann_soap  ann_soanp   xgb_ioa  xgb_soap  xgb_soanp
0  0.872300  0.741169   0.940990  0.988636  0.990013   0.990268
1  0.992121  0.996448   0.995735  0.992227  0.986246   0.986111
2  0.994047  0.992586   0.992216  0.991890  0.989322   0.989270
3  0.998829  0.994716   0.994903  0.997199  0.995715   0.995435
4  0.992200  0.994812   0.994451  0.996243  0.994176   0.995497


In [11]:
df_simple_variance = pd.DataFrame(0.0, index=range(6), columns=['ann_ioa', 'ann_soap', 'ann_soanp', 'xgb_ioa', 'xgb_soap', 'xgb_soanp'])

# Iterate over each row index
for i in range(6):
    # For each column, filter the dataframe, convert to float, and assign the value
    df_simple_variance.loc[i, 'ann_ioa'] = df_simple[df_simple['file_name'].str.contains(f'5306{i*50+50}_ANN_simple_IOA')]['relative_profit'].astype(float).values
    df_simple_variance.loc[i, 'ann_soap'] = df_simple[df_simple['file_name'].str.contains(f'5306{i*50+50}_ANN_simple_SOAp')]['relative_profit'].astype(float).values
    df_simple_variance.loc[i, 'ann_soanp'] = df_simple[df_simple['file_name'].str.contains(f'5306{i*50+50}_ANN_simple_SOAnp')]['relative_profit'].astype(float).values
    df_simple_variance.loc[i, 'xgb_ioa'] = df_simple[df_simple['file_name'].str.contains(f'5306{i*50+50}_XGB_simple_IOA')]['relative_profit'].astype(float).values
    df_simple_variance.loc[i, 'xgb_soap'] = df_simple[df_simple['file_name'].str.contains(f'5306{i*50+50}_XGB_simple_SOAp')]['relative_profit'].astype(float).values
    df_simple_variance.loc[i, 'xgb_soanp'] = df_simple[df_simple['file_name'].str.contains(f'5306{i*50+50}_XGB_simple_SOAnp')]['relative_profit'].astype(float).values

print(df_simple_variance)

    ann_ioa  ann_soap  ann_soanp   xgb_ioa  xgb_soap  xgb_soanp
0  0.992200  0.994812   0.994451  0.996243  0.994176   0.995497
1  0.989853  0.984111   0.985079  0.990344  0.984206   0.983690
2  0.988615  0.982345   0.981734  0.987851  0.978545   0.979641
3  0.986585  0.977765   0.979033  0.986527  0.978802   0.977544
4  0.985893  0.981286   0.979029  0.985561  0.984387   0.983935
5  0.985191  0.977148   0.978496  0.983671  0.971417   0.971127


In [22]:
df_simple_feature_noise = pd.DataFrame(0.0, index=range(7), columns=['ann_ioa', 'ann_soap', 'ann_soanp', 'xgb_ioa', 'xgb_soap', 'xgb_soanp'])

for i in range(7):
    x = 3* (2**i)
    df_simple_feature_noise.loc[i, 'ann_ioa'] = df_simple[df_simple['file_name'].str.contains(f'4{x}0650_ANN_simple_IOA')]['relative_profit'].astype(float).mean()
    df_simple_feature_noise.loc[i, 'ann_soap'] = df_simple[df_simple['file_name'].str.contains(f'4{x}0650_ANN_simple_SOAp')]['relative_profit'].astype(float).mean()
    df_simple_feature_noise.loc[i, 'ann_soanp'] = df_simple[df_simple['file_name'].str.contains(f'4{x}0650_ANN_simple_SOAnp')]['relative_profit'].astype(float).mean()
    df_simple_feature_noise.loc[i, 'xgb_ioa'] = df_simple[df_simple['file_name'].str.contains(f'4{x}0650_XGB_simple_IOA')]['relative_profit'].astype(float).mean()
    df_simple_feature_noise.loc[i, 'xgb_soap'] = df_simple[df_simple['file_name'].str.contains(f'4{x}0650_XGB_simple_SOAp')]['relative_profit'].astype(float).mean()
    df_simple_feature_noise.loc[i, 'xgb_soanp'] = df_simple[df_simple['file_name'].str.contains(f'4{x}0650_XGB_simple_SOAnp')]['relative_profit'].astype(float).mean()

print(df_simple_feature_noise)

    ann_ioa  ann_soap  ann_soanp   xgb_ioa  xgb_soap  xgb_soanp
0  0.998829  0.994716   0.994903  0.997199  0.995715   0.995435
1  0.991578  0.991877   0.992168  0.993388  0.986118   0.985933
2  0.995842  0.994810   0.994009  0.995711  0.989278   0.988921
3  0.995011  0.994816   0.994009  0.994564  0.992827   0.992054
4  0.995041  0.994811   0.994009  0.995518  0.988416   0.988300
5  0.993873  0.994813   0.994009  0.995934  0.992630   0.992281
6  0.993594  0.994797   0.994009  0.995144  0.990580   0.989405


In [ ]:
df_simple_feature_noise = pd.DataFrame(0.0, index=range(7), columns=['ann_ioa', 'ann_soap', 'ann_soanp', 'xgb_ioa', 'xgb_soap', 'xgb_soanp'])

for i in range(7):
    x = 3* (2**i)
    df_simple_feature_noise.loc[i, 'ann_ioa'] = df_simple[df_simple['file_name'].str.contains(f'4{x}1650_ANN_simple_IOA')]['relative_profit'].astype(float).mean()
    df_simple_feature_noise.loc[i, 'ann_soap'] = df_simple[df_simple['file_name'].str.contains(f'4{x}1650_ANN_simple_SOAp')]['relative_profit'].astype(float).mean()
    df_simple_feature_noise.loc[i, 'ann_soanp'] = df_simple[df_simple['file_name'].str.contains(f'4{x}1650_ANN_simple_SOAnp')]['relative_profit'].astype(float).mean()
    df_simple_feature_noise.loc[i, 'xgb_ioa'] = df_simple[df_simple['file_name'].str.contains(f'4{x}1650_XGB_simple_IOA')]['relative_profit'].astype(float).mean()
    df_simple_feature_noise.loc[i, 'xgb_soap'] = df_simple[df_simple['file_name'].str.contains(f'4{x}1650_XGB_simple_SOAp')]['relative_profit'].astype(float).mean()
    df_simple_feature_noise.loc[i, 'xgb_soanp'] = df_simple[df_simple['file_name'].str.contains(f'4{x}1650_XGB_simple_SOAnp')]['relative_profit'].astype(float).mean()

In [13]:
df_simple_feature_noise = pd.DataFrame(0.0, index=range(7), columns=['ann_ioa', 'ann_soap', 'ann_soanp', 'xgb_ioa', 'xgb_soap', 'xgb_soanp'])

df_simple_feature_noise['ann_ioa'][0] = df_simple[df_simple['file_name'].str.contains('430650_ANN_simple_IOA')]['relative_profit'].values
df_simple_feature_noise['ann_ioa'][1] = df_simple[df_simple['file_name'].str.contains('460650_ANN_simple_IOA')]['relative_profit'].values
df_simple_feature_noise['ann_ioa'][2] = df_simple[df_simple['file_name'].str.contains('4120650_ANN_simple_IOA')]['relative_profit'].values
df_simple_feature_noise['ann_ioa'][3] = df_simple[df_simple['file_name'].str.contains('4240650_ANN_simple_IOA')]['relative_profit'].values
df_simple_feature_noise['ann_ioa'][4] = df_simple[df_simple['file_name'].str.contains('4480650_ANN_simple_IOA')]['relative_profit'].values
df_simple_feature_noise['ann_ioa'][5] = df_simple[df_simple['file_name'].str.contains('4960650_ANN_simple_IOA')]['relative_profit'].values
df_simple_feature_noise['ann_ioa'][6] = df_simple[df_simple['file_name'].str.contains('41920650_ANN_simple_IOA')]['relative_profit'].values

df_simple_feature_noise['ann_soap'][0] = df_simple[df_simple['file_name'].str.contains('430650_ANN_simple_SOAp')]['relative_profit'].values
df_simple_feature_noise['ann_soap'][1] = df_simple[df_simple['file_name'].str.contains('460650_ANN_simple_SOAp')]['relative_profit'].values
df_simple_feature_noise['ann_soap'][2] = df_simple[df_simple['file_name'].str.contains('4120650_ANN_simple_SOAp')]['relative_profit'].values
df_simple_feature_noise['ann_soap'][3] = df_simple[df_simple['file_name'].str.contains('4240650_ANN_simple_SOAp')]['relative_profit'].values
df_simple_feature_noise['ann_soap'][4] = df_simple[df_simple['file_name'].str.contains('4480650_ANN_simple_SOAp')]['relative_profit'].values
df_simple_feature_noise['ann_soap'][5] = df_simple[df_simple['file_name'].str.contains('4960650_ANN_simple_SOAp')]['relative_profit'].values
df_simple_feature_noise['ann_soap'][6] = df_simple[df_simple['file_name'].str.contains('41920650_ANN_simple_SOAp')]['relative_profit'].values

df_simple_feature_noise['ann_soanp'][0] = df_simple[df_simple['file_name'].str.contains('430650_ANN_simple_SOAnp')]['relative_profit'].values
df_simple_feature_noise['ann_soanp'][1] = df_simple[df_simple['file_name'].str.contains('460650_ANN_simple_SOAnp')]['relative_profit'].values
df_simple_feature_noise['ann_soanp'][2] = df_simple[df_simple['file_name'].str.contains('4120650_ANN_simple_SOAnp')]['relative_profit'].values
df_simple_feature_noise['ann_soanp'][3] = df_simple[df_simple['file_name'].str.contains('4240650_ANN_simple_SOAnp')]['relative_profit'].values
df_simple_feature_noise['ann_soanp'][4] = df_simple[df_simple['file_name'].str.contains('4480650_ANN_simple_SOAnp')]['relative_profit'].values
df_simple_feature_noise['ann_soanp'][5] = df_simple[df_simple['file_name'].str.contains('4960650_ANN_simple_SOAnp')]['relative_profit'].values
df_simple_feature_noise['ann_soanp'][6] = df_simple[df_simple['file_name'].str.contains('41920650_ANN_simple_SOAnp')]['relative_profit'].values

df_simple_feature_noise['xgb_ioa'][0] = df_simple[df_simple['file_name'].str.contains('430650_XGB_simple_IOA')]['relative_profit'].values
df_simple_feature_noise['xgb_ioa'][1] = df_simple[df_simple['file_name'].str.contains('460650_XGB_simple_IOA')]['relative_profit'].values
df_simple_feature_noise['xgb_ioa'][2] = df_simple[df_simple['file_name'].str.contains('4120650_XGB_simple_IOA')]['relative_profit'].values
df_simple_feature_noise['xgb_ioa'][3] = df_simple[df_simple['file_name'].str.contains('4240650_XGB_simple_IOA')]['relative_profit'].values
df_simple_feature_noise['xgb_ioa'][4] = df_simple[df_simple['file_name'].str.contains('4480650_XGB_simple_IOA')]['relative_profit'].values
df_simple_feature_noise['xgb_ioa'][5] = df_simple[df_simple['file_name'].str.contains('4960650_XGB_simple_IOA')]['relative_profit'].values
df_simple_feature_noise['xgb_ioa'][6] = df_simple[df_simple['file_name'].str.contains('41920650_XGB_simple_IOA')]['relative_profit'].values

df_simple_feature_noise['xgb_soap'][0] = df_simple[df_simple['file_name'].str.contains('430650_XGB_simple_SOAp')]['relative_profit'].values
df_simple_feature_noise['xgb_soap'][1] = df_simple[df_simple['file_name'].str.contains('460650_XGB_simple_SOAp')]['relative_profit'].values
df_simple_feature_noise['xgb_soap'][2] = df_simple[df_simple['file_name'].str.contains('4120650_XGB_simple_SOAp')]['relative_profit'].values
df_simple_feature_noise['xgb_soap'][3] = df_simple[df_simple['file_name'].str.contains('4240650_XGB_simple_SOAp')]['relative_profit'].values
df_simple_feature_noise['xgb_soap'][4] = df_simple[df_simple['file_name'].str.contains('4480650_XGB_simple_SOAp')]['relative_profit'].values
df_simple_feature_noise['xgb_soap'][5] = df_simple[df_simple['file_name'].str.contains('4960650_XGB_simple_SOAp')]['relative_profit'].values
df_simple_feature_noise['xgb_soap'][6] = df_simple[df_simple['file_name'].str.contains('41920650_XGB_simple_SOAp')]['relative_profit'].values

df_simple_feature_noise['xgb_soanp'][0] = df_simple[df_simple['file_name'].str.contains('430650_XGB_simple_SOAnp')]['relative_profit'].values
df_simple_feature_noise['xgb_soanp'][1] = df_simple[df_simple['file_name'].str.contains('460650_XGB_simple_SOAnp')]['relative_profit'].values
df_simple_feature_noise['xgb_soanp'][2] = df_simple[df_simple['file_name'].str.contains('4120650_XGB_simple_SOAnp')]['relative_profit'].values
df_simple_feature_noise['xgb_soanp'][3] = df_simple[df_simple['file_name'].str.contains('4240650_XGB_simple_SOAnp')]['relative_profit'].values
df_simple_feature_noise['xgb_soanp'][4] = df_simple[df_simple['file_name'].str.contains('4480650_XGB_simple_SOAnp')]['relative_profit'].values
df_simple_feature_noise['xgb_soanp'][5] = df_simple[df_simple['file_name'].str.contains('4960650_XGB_simple_SOAnp')]['relative_profit'].values
df_simple_feature_noise['xgb_soanp'][6] = df_simple[df_simple['file_name'].str.contains('41920650_XGB_simple_SOAnp')]['relative_profit'].values

print(df_simple_feature_noise)


    ann_ioa  ann_soap  ann_soanp   xgb_ioa  xgb_soap  xgb_soanp
0  0.998829  0.994716   0.994903  0.997199  0.995715   0.995435
1  0.991578  0.991877   0.992168  0.993388  0.986118   0.985933
2  0.995842  0.994810   0.994009  0.995711  0.989278   0.988921
3  0.995011  0.994816   0.994009  0.994564  0.992827   0.992054
4  0.995041  0.994811   0.994009  0.995518  0.988416   0.988300
5  0.993873  0.994813   0.994009  0.995934  0.992630   0.992281
6  0.993594  0.994797   0.994009  0.995144  0.990580   0.989405


C:\Users\lanza\AppData\Local\Temp\ipykernel_8932\369707523.py:3: FutureWarning: ChainedAssignmentError: behaviour will change in pandas 3.0!
You are setting values through chained assignment. Currently this works in certain cases, but when using Copy-on-Write (which will become the default behaviour in pandas 3.0) this will never work to update the original DataFrame or Series, because the intermediate object on which we are setting values will behave as a copy.
A typical example is when you are setting values in a column of a DataFrame, like:

df["col"][row_indexer] = value

Use `df.loc[row_indexer, "col"] = values` instead, to perform the assignment in a single step and ensure this keeps updating the original `df`.

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

  df_simple_feature_noise['ann_ioa'][0] = df_simple[df_simple['file_name'].str.contains('430650_ANN_simple_IOA')]['relative_profit']

In [ ]:
df_simple_feature_addinfo = pd.DataFrame(0, index=range(8), columns=['ann_ioa', 'ann_soap', 'ann_soanp', 'xgb_ioa', 'xgb_soap', 'xgb_soanp'])

df_simple_feature_addinfo['ann_ioa'][0] = df_simple[df_simple['file_name'].str.contains('431650_ANN_simple_IOA')]['relative_profit'].values
df_simple_feature_addinfo['ann_ioa'][1] = df_simple[df_simple['file_name'].str.contains('461650_ANN_simple_IOA')]['relative_profit'].values
df_simple_feature_addinfo['ann_ioa'][2] = df_simple[df_simple['file_name'].str.contains('4121650_ANN_simple_IOA')]['relative_profit'].values
df_simple_feature_addinfo['ann_ioa'][3] = df_simple[df_simple['file_name'].str.contains('4241650_ANN_simple_IOA')]['relative_profit'].values
df_simple_feature_addinfo['ann_ioa'][4] = df_simple[df_simple['file_name'].str.contains('4481650_ANN_simple_IOA')]['relative_profit'].values
df_simple_feature_addinfo['ann_ioa'][5] = df_simple[df_simple['file_name'].str.contains('4961650_ANN_simple_IOA')]['relative_profit'].values
df_simple_feature_addinfo['ann_ioa'][6] = df_simple[df_simple['file_name'].str.contains('41921650_ANN_simple_IOA')]['relative_profit'].values
df_simple_feature_addinfo['ann_ioa'][7] = df_simple[df_simple['file_name'].str.contains('43841650_ANN_simple_IOA')]['relative_profit'].values

df_simple_feature_addinfo['ann_soap'][0] = df_simple[df_simple['file_name'].str.contains('431650_ANN_simple_SOAp')]['relative_profit'].values
df_simple_feature_addinfo['ann_soap'][1] = df_simple[df_simple['file_name'].str.contains('461650_ANN_simple_SOAp')]['relative_profit'].values
df_simple_feature_addinfo['ann_soap'][2] = df_simple[df_simple['file_name'].str.contains('4121650_ANN_simple_SOAp')]['relative_profit'].values
df_simple_feature_addinfo['ann_soap'][3] = df_simple[df_simple['file_name'].str.contains('4241650_ANN_simple_SOAp')]['relative_profit'].values
df_simple_feature_addinfo['ann_soap'][4] = df_simple[df_simple['file_name'].str.contains('4481650_ANN_simple_SOAp')]['relative_profit'].values
df_simple_feature_addinfo['ann_soap'][5] = df_simple[df_simple['file_name'].str.contains('4961650_ANN_simple_SOAp')]['relative_profit'].values
df_simple_feature_addinfo['ann_soap'][6] = df_simple[df_simple['file_name'].str.contains('41921650_ANN_simple_SOAp')]['relative_profit'].values
df_simple_feature_addinfo['ann_soap'][7] = df_simple[df_simple['file_name'].str.contains('43841650_ANN_simple_SOAp')]['relative_profit'].values

df_simple_feature_addinfo['ann_soanp'][0] = df_simple[df_simple['file_name'].str.contains('431650_ANN_simple_SOAnp')]['relative_profit'].values
df_simple_feature_addinfo['ann_soanp'][1] = df_simple[df_simple['file_name'].str.contains('461650_ANN_simple_SOAnp')]['relative_profit'].values
df_simple_feature_addinfo['ann_soanp'][2] = df_simple[df_simple['file_name'].str.contains('4121650_ANN_simple_SOAnp')]['relative_profit'].values
df_simple_feature_addinfo['ann_soanp'][3] = df_simple[df_simple['file_name'].str.contains('4241650_ANN_simple_SOAnp')]['relative_profit'].values
df_simple_feature_addinfo['ann_soanp'][4] = df_simple[df_simple['file_name'].str.contains('4481650_ANN_simple_SOAnp')]['relative_profit'].values
df_simple_feature_addinfo['ann_soanp'][5] = df_simple[df_simple['file_name'].str.contains('4961650_ANN_simple_SOAnp')]['relative_profit'].values
df_simple_feature_addinfo['ann_soanp'][6] = df_simple[df_simple['file_name'].str.contains('41921650_ANN_simple_SOAnp')]['relative_profit'].values
df_simple_feature_addinfo['ann_soanp'][7] = df_simple[df_simple['file_name'].str.contains('43841650_ANN_simple_SOAnp')]['relative_profit'].values

df_simple_feature_addinfo['xgb_ioa'][0] = df_simple[df_simple['file_name'].str.contains('431650_XGB_simple_IOA')]['relative_profit'].values
df_simple_feature_addinfo['xgb_ioa'][1] = df_simple[df_simple['file_name'].str.contains('461650_XGB_simple_IOA')]['relative_profit'].values
df_simple_feature_addinfo['xgb_ioa'][2] = df_simple[df_simple['file_name'].str.contains('4121650_XGB_simple_IOA')]['relative_profit'].values
df_simple_feature_addinfo['xgb_ioa'][3] = df_simple[df_simple['file_name'].str.contains('4241650_XGB_simple_IOA')]['relative_profit'].values
df_simple_feature_addinfo['xgb_ioa'][4] = df_simple[df_simple['file_name'].str.contains('4481650_XGB_simple_IOA')]['relative_profit'].values
df_simple_feature_addinfo['xgb_ioa'][5] = df_simple[df_simple['file_name'].str.contains('4961650_XGB_simple_IOA')]['relative_profit'].values
df_simple_feature_addinfo['xgb_ioa'][6] = df_simple[df_simple['file_name'].str.contains('41921650_XGB_simple_IOA')]['relative_profit'].values
df_simple_feature_addinfo['xgb_ioa'][7] = df_simple[df_simple['file_name'].str.contains('43841650_XGB_simple_IOA')]['relative_profit'].values

df_simple_feature_addinfo['xgb_soap'][0] = df_simple[df_simple['file_name'].str.contains('431650_XGB_simple_SOAp')]['relative_profit'].values
df_simple_feature_addinfo['xgb_soap'][1] = df_simple[df_simple['file_name'].str.contains('461650_XGB_simple_SOAp')]['relative_profit'].values
df_simple_feature_addinfo['xgb_soap'][2] = df_simple[df_simple['file_name'].str.contains('4121650_XGB_simple_SOAp')]['relative_profit'].values
df_simple_feature_addinfo['xgb_soap'][3] = df_simple[df_simple['file_name'].str.contains('4241650_XGB_simple_SOAp')]['relative_profit'].values
df_simple_feature_addinfo['xgb_soap'][4] = df_simple[df_simple['file_name'].str.contains('4481650_XGB_simple_SOAp')]['relative_profit'].values
df_simple_feature_addinfo['xgb_soap'][5] = df_simple[df_simple['file_name'].str.contains('4961650_XGB_simple_SOAp')]['relative_profit'].values
df_simple_feature_addinfo['xgb_soap'][6] = df_simple[df_simple['file_name'].str.contains('41921650_XGB_simple_SOAp')]['relative_profit'].values
df_simple_feature_addinfo['xgb_soap'][7] = df_simple[df_simple['file_name'].str.contains('43841650_XGB_simple_SOAp')]['relative_profit'].values

df_simple_feature_addinfo['xgb_soanp'][0] = df_simple[df_simple['file_name'].str.contains('431650_XGB_simple_SOAnp')]['relative_profit'].values
df_simple_feature_addinfo['xgb_soanp'][1] = df_simple[df_simple['file_name'].str.contains('461650_XGB_simple_SOAnp')]['relative_profit'].values
df_simple_feature_addinfo['xgb_soanp'][2] = df_simple[df_simple['file_name'].str.contains('4121650_XGB_simple_SOAnp')]['relative_profit'].values
df_simple_feature_addinfo['xgb_soanp'][3] = df_simple[df_simple['file_name'].str.contains('4241650_XGB_simple_SOAnp')]['relative_profit'].values
df_simple_feature_addinfo['xgb_soanp'][4] = df_simple[df_simple['file_name'].str.contains('4481650_XGB_simple_SOAnp')]['relative_profit'].values
df_simple_feature_addinfo['xgb_soanp'][5] = df_simple[df_simple['file_name'].str.contains('4961650_XGB_simple_SOAnp')]['relative_profit'].values
df_simple_feature_addinfo['xgb_soanp'][6] = df_simple[df_simple['file_name'].str.contains('41921650_XGB_simple_SOAnp')]['relative_profit'].values
df_simple_feature_addinfo['xgb_soanp'][7] = df_simple[df_simple['file_name'].str.contains('43841650_XGB_simple_SOAnp')]['relative_profit'].values

print(df_simple_feature_addinfo)


    ann_ioa  ann_soap  ann_soanp   xgb_ioa  xgb_soap  xgb_soanp
0  0.980734  0.977477   0.979000  0.983421  0.979809   0.979379
1  0.983562  0.974038   0.977361  0.983530  0.971002   0.970321
2  0.983172  0.976895   0.979573  0.981807  0.958167   0.958715
3  0.980752  0.977183   0.979279  0.982614  0.965231   0.964981
4  0.979095  0.977223   0.977839  0.982347  0.959647   0.957011
5  0.980957  0.977086   0.979103  0.980145  0.967988   0.967823
6  0.974475  0.974077   0.975176  0.981120  0.972237   0.972441
7  0.976536  0.964164   0.967297  0.979804  0.962150   0.961526


C:\Users\lanza\AppData\Local\Temp\ipykernel_28796\3434342612.py:3: FutureWarning: ChainedAssignmentError: behaviour will change in pandas 3.0!
You are setting values through chained assignment. Currently this works in certain cases, but when using Copy-on-Write (which will become the default behaviour in pandas 3.0) this will never work to update the original DataFrame or Series, because the intermediate object on which we are setting values will behave as a copy.
A typical example is when you are setting values in a column of a DataFrame, like:

df["col"][row_indexer] = value

Use `df.loc[row_indexer, "col"] = values` instead, to perform the assignment in a single step and ensure this keeps updating the original `df`.

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

  df_simple_feature_addinfo['ann_ioa'][0] = df_simple[df_simple['file_name'].str.contains('431650_ANN_simple_IOA')]['relative_prof

In [ ]:
df_simple_heterogenity = pd.DataFrame(0, index=range(6), columns=['ann_ioa', 'ann_soap', 'ann_soanp', 'xgb_ioa', 'xgb_soap', 'xgb_soanp'])

df_simple_heterogenity['ann_ioa'][0] = df_simple[df_simple['file_name'].str.contains('430650_ANN_simple_IOA')]['relative_profit'].values
df_simple_heterogenity['ann_ioa'][1] = df_simple[df_simple['file_name'].str.contains('4306510_ANN_simple_IOA')]['relative_profit'].values
df_simple_heterogenity['ann_ioa'][2] = df_simple[df_simple['file_name'].str.contains('4306520_ANN_simple_IOA')]['relative_profit'].values
df_simple_heterogenity['ann_ioa'][3] = df_simple[df_simple['file_name'].str.contains('4306530_ANN_simple_IOA')]['relative_profit'].values
df_simple_heterogenity['ann_ioa'][4] = df_simple[df_simple['file_name'].str.contains('4306540_ANN_simple_IOA')]['relative_profit'].values
df_simple_heterogenity['ann_ioa'][5] = df_simple[df_simple['file_name'].str.contains('4306550_ANN_simple_IOA')]['relative_profit'].values

df_simple_heterogenity['ann_soap'][0] = df_simple[df_simple['file_name'].str.contains('430650_ANN_simple_SOAp')]['relative_profit'].values
df_simple_heterogenity['ann_soap'][1] = df_simple[df_simple['file_name'].str.contains('4306510_ANN_simple_SOAp')]['relative_profit'].values
df_simple_heterogenity['ann_soap'][2] = df_simple[df_simple['file_name'].str.contains('4306520_ANN_simple_SOAp')]['relative_profit'].values
df_simple_heterogenity['ann_soap'][3] = df_simple[df_simple['file_name'].str.contains('4306530_ANN_simple_SOAp')]['relative_profit'].values
df_simple_heterogenity['ann_soap'][4] = df_simple[df_simple['file_name'].str.contains('4306540_ANN_simple_SOAp')]['relative_profit'].values
df_simple_heterogenity['ann_soap'][5] = df_simple[df_simple['file_name'].str.contains('4306550_ANN_simple_SOAp')]['relative_profit'].values

df_simple_heterogenity['ann_soanp'][0] = df_simple[df_simple['file_name'].str.contains('430650_ANN_simple_SOAnp')]['relative_profit'].values
df_simple_heterogenity['ann_soanp'][1] = df_simple[df_simple['file_name'].str.contains('4306510_ANN_simple_SOAnp')]['relative_profit'].values
df_simple_heterogenity['ann_soanp'][2] = df_simple[df_simple['file_name'].str.contains('4306520_ANN_simple_SOAnp')]['relative_profit'].values
df_simple_heterogenity['ann_soanp'][3] = df_simple[df_simple['file_name'].str.contains('4306530_ANN_simple_SOAnp')]['relative_profit'].values
df_simple_heterogenity['ann_soanp'][4] = df_simple[df_simple['file_name'].str.contains('4306540_ANN_simple_SOAnp')]['relative_profit'].values
df_simple_heterogenity['ann_soanp'][5] = df_simple[df_simple['file_name'].str.contains('4306550_ANN_simple_SOAnp')]['relative_profit'].values

df_simple_heterogenity['xgb_ioa'][0] = df_simple[df_simple['file_name'].str.contains('430650_XGB_simple_IOA')]['relative_profit'].values
df_simple_heterogenity['xgb_ioa'][1] = df_simple[df_simple['file_name'].str.contains('4306510_XGB_simple_IOA')]['relative_profit'].values
df_simple_heterogenity['xgb_ioa'][2] = df_simple[df_simple['file_name'].str.contains('4306520_XGB_simple_IOA')]['relative_profit'].values
df_simple_heterogenity['xgb_ioa'][3] = df_simple[df_simple['file_name'].str.contains('4306530_XGB_simple_IOA')]['relative_profit'].values
df_simple_heterogenity['xgb_ioa'][4] = df_simple[df_simple['file_name'].str.contains('4306540_XGB_simple_IOA')]['relative_profit'].values
df_simple_heterogenity['xgb_ioa'][5] = df_simple[df_simple['file_name'].str.contains('4306550_XGB_simple_IOA')]['relative_profit'].values

df_simple_heterogenity['xgb_soap'][0] = df_simple[df_simple['file_name'].str.contains('430650_XGB_simple_SOAp')]['relative_profit'].values
df_simple_heterogenity['xgb_soap'][1] = df_simple[df_simple['file_name'].str.contains('4306510_XGB_simple_SOAp')]['relative_profit'].values
df_simple_heterogenity['xgb_soap'][2] = df_simple[df_simple['file_name'].str.contains('4306520_XGB_simple_SOAp')]['relative_profit'].values
df_simple_heterogenity['xgb_soap'][3] = df_simple[df_simple['file_name'].str.contains('4306530_XGB_simple_SOAp')]['relative_profit'].values
df_simple_heterogenity['xgb_soap'][4] = df_simple[df_simple['file_name'].str.contains('4306540_XGB_simple_SOAp')]['relative_profit'].values
df_simple_heterogenity['xgb_soap'][5] = df_simple[df_simple['file_name'].str.contains('4306550_XGB_simple_SOAp')]['relative_profit'].values

df_simple_heterogenity['xgb_soanp'][0] = df_simple[df_simple['file_name'].str.contains('430650_XGB_simple_SOAnp')]['relative_profit'].values
df_simple_heterogenity['xgb_soanp'][1] = df_simple[df_simple['file_name'].str.contains('4306510_XGB_simple_SOAnp')]['relative_profit'].values
df_simple_heterogenity['xgb_soanp'][2] = df_simple[df_simple['file_name'].str.contains('4306520_XGB_simple_SOAnp')]['relative_profit'].values
df_simple_heterogenity['xgb_soanp'][3] = df_simple[df_simple['file_name'].str.contains('4306530_XGB_simple_SOAnp')]['relative_profit'].values
df_simple_heterogenity['xgb_soanp'][4] = df_simple[df_simple['file_name'].str.contains('4306540_XGB_simple_SOAnp')]['relative_profit'].values
df_simple_heterogenity['xgb_soanp'][5] = df_simple[df_simple['file_name'].str.contains('4306550_XGB_simple_SOAnp')]['relative_profit'].values

print(df_simple_heterogenity)


    ann_ioa  ann_soap  ann_soanp   xgb_ioa  xgb_soap  xgb_soanp
0  0.998829  0.994716   0.994903  0.997199  0.995715   0.995435
1  0.992119  0.989059   0.989170  0.990650  0.987774   0.986892
2  0.991440  0.991385   0.990934  0.991617  0.988732   0.989731
3  0.992491  0.977971   0.978797  0.991920  0.983775   0.984234
4  0.990721  0.990298   0.989862  0.991359  0.984197   0.984225
5  0.991970  0.990712   0.990723  0.991741  0.986952   0.987289


C:\Users\lanza\AppData\Local\Temp\ipykernel_28796\3849460272.py:3: FutureWarning: ChainedAssignmentError: behaviour will change in pandas 3.0!
You are setting values through chained assignment. Currently this works in certain cases, but when using Copy-on-Write (which will become the default behaviour in pandas 3.0) this will never work to update the original DataFrame or Series, because the intermediate object on which we are setting values will behave as a copy.
A typical example is when you are setting values in a column of a DataFrame, like:

df["col"][row_indexer] = value

Use `df.loc[row_indexer, "col"] = values` instead, to perform the assignment in a single step and ensure this keeps updating the original `df`.

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

  df_simple_heterogenity['ann_ioa'][0] = df_simple[df_simple['file_name'].str.contains('430650_ANN_simple_IOA')]['relative_profit'

In [ ]:
df_size_complex = pd.DataFrame(0, index=range(5), columns=['ann_ioa', 'ann_soap', 'ann_soanp', 'xgb_ioa', 'xgb_soap', 'xgb_soanp'])

df_size_complex['ann_ioa'][0] = df_complex[df_complex['file_name'].str.contains('130650_ANN_complex_IOA')]['relative_profit'].values
df_size_complex['ann_ioa'][1] = df_complex[df_complex['file_name'].str.contains('230650_ANN_complex_IOA')]['relative_profit'].values
df_size_complex['ann_ioa'][2] = df_complex[df_complex['file_name'].str.contains('330650_ANN_complex_IOA')]['relative_profit'].values
df_size_complex['ann_ioa'][3] = df_complex[df_complex['file_name'].str.contains('430650_ANN_complex_IOA')]['relative_profit'].values
df_size_complex['ann_ioa'][4] = df_complex[df_complex['file_name'].str.contains('530650_ANN_complex_IOA')]['relative_profit'].values

df_size_complex['ann_soap'][0] = df_complex[df_complex['file_name'].str.contains('130650_ANN_complex_SOAp')]['relative_profit'].values
df_size_complex['ann_soap'][1] = df_complex[df_complex['file_name'].str.contains('230650_ANN_complex_SOAp')]['relative_profit'].values
df_size_complex['ann_soap'][2] = df_complex[df_complex['file_name'].str.contains('330650_ANN_complex_SOAp')]['relative_profit'].values
df_size_complex['ann_soap'][3] = df_complex[df_complex['file_name'].str.contains('430650_ANN_complex_SOAp')]['relative_profit'].values
df_size_complex['ann_soap'][4] = df_complex[df_complex['file_name'].str.contains('530650_ANN_complex_SOAp')]['relative_profit'].values

df_size_complex['ann_soanp'][0] = df_complex[df_complex['file_name'].str.contains('130650_ANN_complex_SOAnp')]['relative_profit'].values
df_size_complex['ann_soanp'][1] = df_complex[df_complex['file_name'].str.contains('230650_ANN_complex_SOAnp')]['relative_profit'].values
df_size_complex['ann_soanp'][2] = df_complex[df_complex['file_name'].str.contains('330650_ANN_complex_SOAnp')]['relative_profit'].values
df_size_complex['ann_soanp'][3] = df_complex[df_complex['file_name'].str.contains('430650_ANN_complex_SOAnp')]['relative_profit'].values
df_size_complex['ann_soanp'][4] = df_complex[df_complex['file_name'].str.contains('530650_ANN_complex_SOAnp')]['relative_profit'].values

df_size_complex['xgb_ioa'][0] = df_complex[df_complex['file_name'].str.contains('130650_XGB_complex_IOA')]['relative_profit'].values
df_size_complex['xgb_ioa'][1] = df_complex[df_complex['file_name'].str.contains('230650_XGB_complex_IOA')]['relative_profit'].values
df_size_complex['xgb_ioa'][2] = df_complex[df_complex['file_name'].str.contains('330650_XGB_complex_IOA')]['relative_profit'].values
df_size_complex['xgb_ioa'][3] = df_complex[df_complex['file_name'].str.contains('430650_XGB_complex_IOA')]['relative_profit'].values
df_size_complex['xgb_ioa'][4] = df_complex[df_complex['file_name'].str.contains('530650_XGB_complex_IOA')]['relative_profit'].values

df_size_complex['xgb_soap'][0] = df_complex[df_complex['file_name'].str.contains('130650_XGB_complex_SOAp')]['relative_profit'].values
df_size_complex['xgb_soap'][1] = df_complex[df_complex['file_name'].str.contains('230650_XGB_complex_SOAp')]['relative_profit'].values
df_size_complex['xgb_soap'][2] = df_complex[df_complex['file_name'].str.contains('330650_XGB_complex_SOAp')]['relative_profit'].values
df_size_complex['xgb_soap'][3] = df_complex[df_complex['file_name'].str.contains('430650_XGB_complex_SOAp')]['relative_profit'].values
df_size_complex['xgb_soap'][4] = df_complex[df_complex['file_name'].str.contains('530650_XGB_complex_SOAp')]['relative_profit'].values

df_size_complex['xgb_soanp'][0] = df_complex[df_complex['file_name'].str.contains('130650_XGB_complex_SOAnp')]['relative_profit'].values
df_size_complex['xgb_soanp'][1] = df_complex[df_complex['file_name'].str.contains('230650_XGB_complex_SOAnp')]['relative_profit'].values
df_size_complex['xgb_soanp'][2] = df_complex[df_complex['file_name'].str.contains('330650_XGB_complex_SOAnp')]['relative_profit'].values
df_size_complex['xgb_soanp'][3] = df_complex[df_complex['file_name'].str.contains('430650_XGB_complex_SOAnp')]['relative_profit'].values
df_size_complex['xgb_soanp'][4] = df_complex[df_complex['file_name'].str.contains('530650_XGB_complex_SOAnp')]['relative_profit'].values

print(df_size_complex)

C:\Users\lanza\AppData\Local\Temp\ipykernel_28796\2487695880.py:3: FutureWarning: ChainedAssignmentError: behaviour will change in pandas 3.0!
You are setting values through chained assignment. Currently this works in certain cases, but when using Copy-on-Write (which will become the default behaviour in pandas 3.0) this will never work to update the original DataFrame or Series, because the intermediate object on which we are setting values will behave as a copy.
A typical example is when you are setting values in a column of a DataFrame, like:

df["col"][row_indexer] = value

Use `df.loc[row_indexer, "col"] = values` instead, to perform the assignment in a single step and ensure this keeps updating the original `df`.

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

  df_size_complex['ann_ioa'][0] = df_complex[df_complex['file_name'].str.contains('130650_ANN_complex_IOA')]['relative_profit'].va

    ann_ioa   ann_soap  ann_soanp   xgb_ioa   xgb_soap  xgb_soanp
0  0.827391   0.204170   0.961835  0.995812   0.993212   0.993212
1  0.000000   0.009898   0.009893  0.994567  98.890873  98.890886
2  0.000000   0.009915   0.009918  0.995921   0.009920   0.009920
3  0.996466  99.273076  99.283132  0.996593  99.221756  99.212965
4       NaN   0.009926   0.009925  0.995591  99.239372  99.236043
